In [2]:
from __future__ import print_function
from ortools.sat.python import cp_model
import time
import numpy as np


In [8]:
# define model: use CpModel()
model = cp_model.CpModel()

start = time.time()
impact = [
    [30, 45, 40, 50, 70, 90, 110, 100],
    [35, 50, 45, 66, 79, 100, 130, 90],
    [40, 60, 50, 70, 80, 90, 100, 90],
    [55, 65, 60, 80, 90, 110, 60, 80],
    [79, 80, 60, 90, 105, 120, 80, 97],
    [86, 90, 80, 95, 110, 130, 90, 100],
    [90, 100, 90, 148, 103, 140, 95, 110],
    [180, 190, 110, 159, 116, 190, 140, 130],
    [200, 260, 180, 170, 150, 200, 180, 190],
    [200, 270, 260, 200, 170, 210, 200, 190]]

# Variables
num_vehicles = len(impact)
num_deliveries = len(impact[0])

# Defining the groups
group_1 = [0, 1, 2, 3]
group_2 = [4, 5, 6]
group_3 = [7, 8, 9]

# Defining autonomy / range of each group
group_1_autonomy = 150
group_2_autonomy = 300
group_3_autonomy = 500

distance = [130, 80, 230, 100, 150, 440, 110, 250]

x = {}
for i in range(num_vehicles):
    for j in range(num_deliveries):
        x[i, j] = model.NewIntVar(0, 1, '')

# Constraints

# Each delivery must be served by exactly one vehicle
# NB! Use small 's' (sum) for the CP model(!)
# NB! We dont use model.sum - just sum when adding constraints
# NB! We dont have square brackets around the x and the range lik in Assignment Problems using CLP solver
# =! solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

for j in range(num_deliveries):
    model.Add(sum(x[i, j] for i in range(num_vehicles)) == 1)

# Each vehicle can have maximum one delivery
for i in range(num_vehicles):
    model.Add(sum(x[i, j] for j in range(num_deliveries)) <= 1)

# Adding maximum autonomy for the different v. groups
# Can we take all in one - Yes
# for j in range(num_deliveries):
#     model.Add(sum((distance[j] * x[i, j])
#               for i in group_1) <= group_1_autonomy)
#
# for j in range(num_deliveries):
#     model.Add(sum((distance[j] * x[i, j])
#               for i in group_2) <= group_2_autonomy)
#
# for j in range(num_deliveries):
#     model.Add(sum((distance[j] * x[i, j])
#               for i in group_3) <= group_3_autonomy)

for j in range(num_deliveries):
    model.Add(sum((distance[j] * x[i, j])
              for i in group_1) <= group_1_autonomy)
    model.Add(sum((distance[j] * x[i, j])
              for i in group_2) <= group_2_autonomy)
    model.Add(sum((distance[j] * x[i, j])
              for i in group_3) <= group_3_autonomy)

# define objective function and solve the model

model.Minimize(sum([impact[i][j] * x[i, j]
               for i in range(num_vehicles) for j in range(num_deliveries)]))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Minimum impact = %i' % solver.ObjectiveValue())
    print()

    for i in range(num_vehicles):
        for j in range(num_deliveries):
            try:
                # if solver.Value(x[i][j]) == 1: <--- This was the code, changed to the line below. Runs like a charm.
                if solver.Value(x[i, j]) == 1:
                    print('Vehicle ', i, ' performs delivery ',
                          j, '  Km = ', impact[i][j])
            except:
                print('Error')
    print()
    end = time.time()
    print("Time = ", round(end - start, 4), "seconds")


Minimum impact = 653

Vehicle  0  performs delivery  3   Km =  50
Vehicle  1  performs delivery  1   Km =  50
Vehicle  2  performs delivery  0   Km =  40
Vehicle  3  performs delivery  6   Km =  60
Vehicle  4  performs delivery  2   Km =  60
Vehicle  5  performs delivery  7   Km =  100
Vehicle  6  performs delivery  4   Km =  103
Vehicle  7  performs delivery  5   Km =  190

Time =  0.0115 seconds
